<a href="https://colab.research.google.com/github/tboyea/2023-Math-152/blob/main/Exploration_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploration 2: Newton's method and basins of attraction

By: Tanner Boyea, Kelly Kirchner, Samuel Neff

Newton's Method is an algorithm used to approximate the roots of a differentiable function.

The method goes as followed:

First, you start with any guess as long as the function's derivative at that point is non-zero. We can call this guess $x_n$.
Then, you calculate the functions value and derivative at that point, we can call them $f(x_n)$ and $f'(x_n)$.
After this, you find the next term by $x_{n+1} = x - \frac{f(x_n)}{f'(x_n)}$.
We can epeat this step until sufficient enough precision.

#Tools:
First, we will need to import all packages we plan on using
Numpy will be used to multiply lists and initialize the array of pixels, 
matplotlib.pyplot will be used to visualize the steps of convergence, 
PIL will be used to visualize the basins of attraction.

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
     

Newton's Fractal Setup

In [ ]:
import numpy as np
import random as rand

#-----------------------------COLORS-----------------------------

colorList = [ [0.0   , 0.4470, 0.7410],\
              [0.8500, 0.3250, 0.0980],\
              [0.9290, 0.6940, 0.1250],\
              [0.4660, 0.6740, 0.1880],\
              [0.3010, 0.7450, 0.9330],\
              [0.6350, 0.0780, 0.1840] ]

def getColor(n):
  # returns nth color from colorlist. if n geq len, return random color 
  if (n < len(colorList)):
      return colorList(n)
  else:
      return [rand.random(), rand.random(), rand.random()]


#-----------------------------MATH-----------------------------

# note: ik these are all functions in numpy.
def polyDeriv(coeffs):
  # returns the derivative of a polynomial whose nth degree coefficients are coeffs[n]
  # output is again a list of coefficents
  return [coeffs[n+1]*(n+1) for n in range(0,len(coeffs)-1)]

def polyEval(coeffs, X):
  # evaluates a polynomial at an array of numpy points X
  out = np.zeros(np.size(X))
  for n in range(0,len(coeffs)):
      out += np.multiply(np.power(X,n), coeffs[n])
  return out

def polyToString(coeffs):
  # makes stuff readable
  out = ""
  for n in range(len(coeffs)-1,-1,-1):
      out += str(coeffs[n]) + "x^" + str(n)
      if (n>0): out += " + "
  return out  


In [ ]:
print(polyEval([0,1,2,3],np.array([3,4])))
print(polyToString([0,1,2,3]))
print(polyToString(polyDeriv([0,1,2,3])))

[102. 228.]
3x^3 + 2x^2 + 1x^1 + 0x^0
9x^2 + 4x^1 + 1x^0


In [ ]:
def newtFractal(coeffs):
  

In [ ]:
def polyNewtMethod(coeffs, X, itterations):
    # newtons method for arbitrary numpy inputs
    # note we use a for-loop rather than a recursive scheme because we expect the sequence for different values of X to converge at wildly different rates
    for n in range(1,itterations):
    